In [1]:
#Import de pandas
import pandas as pd

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/aaudric/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aaudric/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
#Chargement des données
data = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_sport_analyse.csv")
data2 = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_fasting_analyse.csv")
data3 = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_cannabis.csv")
tweets_df = pd.concat([data,data2,data3])

In [5]:
tweets_df.head()

,Fichier,ID,Nb retweet,Nb like,Nb réponses,Nb citations,Hashtags,Texte,Mots,Phrases,Tags POS,Sentiment,Polarité,Subjectivité,Phrases nominales,Texte corrigé,Texte tokénizé
0,721916579533950976.json,721916579533950976,0,0,0,0.0,"['#CANCER', '#SPORT', '#PARENTING', '#SCHOOL',...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet...","[Sentence(""Is artificial turf giving athletes ...","[('Is', 'VBZ'), ('artificial', 'JJ'), ('turf',...","Sentiment(polarity=-0.6, subjectivity=1.0)",-0.6,1.0000,"['artificial turf', 'athletes # cancer', 'spor...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet..."
1,238304774825603072.json,238304774825603072,0,1,0,0.0,"['#CANCER', '#SPORT']",Frankel's victory at York made me feel '20 yea...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma...","[Sentence(""Frankel's victory at York made me f...","[('Frankel', 'NNP'), (""'s"", 'POS'), ('victory'...","Sentiment(polarity=0.5, subjectivity=0.5)",0.5,0.5000,"['frankel', ""'s victory"", 'york', ""'20 years"",...",Fraenkel's victory at Work made me feel '20 ye...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma..."
2,1007390700726575104.json,1007390700726575104,0,4,1,0.0,"['#SPORT', '#CANCER']",READ THIS! What an absolutely brilliant piece ...,"['READ', 'THIS', 'What', 'an', 'absolutely', '...","[Sentence(""READ THIS!""), Sentence(""What an abs...","[('READ', 'VB'), ('THIS', 'NN'), ('What', 'WP'...","Sentiment(polarity=0.5, subjectivity=0.5375)",0.5,0.5375,"['read this', 'brilliant piece', '# sport trib...",READ THIS! That an absolutely brilliant piece ...,"['READ', 'THIS', '!', 'What', 'an', 'absolutel..."
3,668965073889697795.json,668965073889697795,0,0,0,0.0,"['#CANCER', '#STROKE', '#SPORT']","MJA InSight News: #Cancer survivors, exercise ...","['MJA', 'InSight', 'News', 'Cancer', 'survivor...","[Sentence(""MJA InSight News: #Cancer survivors...","[('MJA', 'NNP'), ('InSight', 'NNP'), ('News', ...","Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0000,"['mja insight', '# cancer survivors', 'atopic ...","MJA night News: #Cancer survivor, exercise bar...","['MJA', 'InSight', 'News', ':', '#', 'Cancer',..."
4,1098650711049555968.json,1098650711049555968,2,1,0,0.0,"['#HEALTH', '#CANCER', '#SPORT']",When #health #cancer &amp; #sport are inextric...,"['When', 'health', 'cancer', 'amp', 'sport', '...","[Sentence(""When #health #cancer &amp; #sport a...","[('When', 'WRB'), ('health', 'NN'), ('cancer',...","Sentiment(polarity=0.6000000000000001, subject...",0.6,0.9000,"['# health # cancer', '# sport', 'amazing', 'd...",When #health #cancer &amp; #sport are inextric...,"['When', '#', 'health', '#', 'cancer', '&', 'a..."


In [6]:
#La racinisation (stemming) consiste à transformer les mots en forme radicale c’est ce que fait la fonction SnowballStemmer de Nltk.
stemmer = SnowballStemmer('english') #ici anglais car nos tweets sont en anglais

In [7]:
#Toutes les lignes de la colonne "Texte corrigé" n'étaient pas toutes de types 'str'
tweets_df['Texte corrigé'] = tweets_df['Texte corrigé'].astype(str)

#Suppression des liens commençant par http et https  
def remove_url(text):
 return re.sub(r'https?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url)
def remove_url1(text):
 return re.sub(r'http?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url1)

In [8]:
#opération de mettre les mots sous leur forme canonique 
#ex: running devient run 
def lemmatize_stemming(text) :
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))

In [9]:
#suppression des stopwords (the, or, and…) et des mots de moins de 3 lettres
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


# Imprimez le résultat intermédiaire pour chaque document
processed_docs = [preprocess(doc) for doc in tweets_df]
#for doc in processed_docs:
    # Concaténez les mots pour former une phrase
    #doc_text = ' '.join(doc)
    #print(preprocess(doc_text))
    



In [10]:
# On stocke les données dans un dictionnaire Gensim et on les convertit au format Bag Of Words(bow).
#On a alors un couple mot/nb d'occurence

# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Créez le dictionnaire et le corpus
dictionary = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Créez et entraînez le modèle LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=15, # Le nombre de topics à extraire
                                       id2word=dictionary, # Le mapping entre les identifiants et les mots
                                       passes=50) # Le nombre d'itérations dans le corpus/ passages




# Affichez les topics
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

Topic: 0 -> Words: 0.061*"cancer" + 0.047*"pain" + 0.044*"connai" + 0.035*"health" + 0.025*"anxieti" + 0.021*"hemp" + 0.016*"depress" + 0.014*"cannabi" + 0.014*"boil" + 0.011*"medicin"
Topic: 1 -> Words: 0.154*"cancer" + 0.105*"connai" + 0.096*"marijuana" + 0.036*"pushkin" + 0.030*"donald" + 0.029*"lung" + 0.026*"hemp" + 0.025*"hung" + 0.024*"epilepsi" + 0.024*"associ"
Topic: 2 -> Words: 0.198*"cancer" + 0.162*"marijuana" + 0.114*"connai" + 0.069*"medic" + 0.042*"cannabi" + 0.032*"hemp" + 0.031*"help" + 0.023*"despoil" + 0.019*"vote" + 0.018*"child"
Topic: 3 -> Words: 0.046*"cancer" + 0.040*"cannabi" + 0.022*"plant" + 0.020*"stop" + 0.017*"food" + 0.017*"diseas" + 0.014*"aid" + 0.013*"extract" + 0.013*"good" + 0.013*"wietoli"
Topic: 4 -> Words: 0.113*"cancer" + 0.103*"connai" + 0.067*"cell" + 0.056*"kill" + 0.034*"cannabinoid" + 0.033*"corner" + 0.025*"weed" + 0.017*"thou" + 0.016*"effect" + 0.015*"research"
Topic: 5 -> Words: 0.084*"cancer" + 0.070*"connai" + 0.023*"thank" + 0.023*"la

In [11]:
#Mesure de la cohérence du modèle
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.2789641086853283


In [12]:
#On garde les résultats dans un df
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [13]:
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.061, cancer)","(0.047, pain)","(0.044, connai)","(0.035, health)","(0.025, anxieti)","(0.021, hemp)","(0.016, depress)","(0.014, cannabi)","(0.014, boil)","(0.011, medicin)"
Topic 2,"(0.154, cancer)","(0.105, connai)","(0.096, marijuana)","(0.036, pushkin)","(0.030, donald)","(0.029, lung)","(0.026, hemp)","(0.025, hung)","(0.024, epilepsi)","(0.024, associ)"
Topic 3,"(0.198, cancer)","(0.162, marijuana)","(0.114, connai)","(0.069, medic)","(0.042, cannabi)","(0.032, hemp)","(0.031, help)","(0.023, despoil)","(0.019, vote)","(0.018, child)"
Topic 4,"(0.046, cancer)","(0.040, cannabi)","(0.022, plant)","(0.020, stop)","(0.017, food)","(0.017, diseas)","(0.014, aid)","(0.013, extract)","(0.013, good)","(0.013, wietoli)"
Topic 5,"(0.113, cancer)","(0.103, connai)","(0.067, cell)","(0.056, kill)","(0.034, cannabinoid)","(0.033, corner)","(0.025, weed)","(0.017, thou)","(0.016, effect)","(0.015, research)"
Topic 6,"(0.084, cancer)","(0.070, connai)","(0.023, thank)","(0.023, latest)","(0.023, medic)","(0.023, support)","(0.016, news)","(0.015, free)","(0.015, sport)","(0.013, best)"
Topic 7,"(0.153, cancer)","(0.110, cannabinoid)","(0.092, connai)","(0.086, tumor)","(0.062, growth)","(0.054, cannabi)","(0.035, inhibit)","(0.035, treatment)","(0.027, marijuana)","(0.025, reduc)"
Topic 8,"(0.109, cancer)","(0.069, marijuana)","(0.063, cannabi)","(0.047, connai)","(0.043, cure)","(0.032, medicin)","(0.020, health)","(0.018, despoil)","(0.017, biochemist)","(0.015, medicalcannabi)"
Topic 9,"(0.086, cancer)","(0.055, connai)","(0.048, marijuana)","(0.030, cannabi)","(0.025, epilepsi)","(0.025, day)","(0.022, hemp)","(0.017, life)","(0.016, save)","(0.016, patient)"
Topic 10,"(0.095, cancer)","(0.086, connai)","(0.028, help)","(0.021, legal)","(0.021, cannabi)","(0.020, weed)","(0.020, like)","(0.019, peopl)","(0.017, news)","(0.016, want)"


In [14]:
import pyLDAvis.gensim_models

In [15]:
#Visu interractive affichant les topics, les mots etc
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.109576  0.119085       1        1  14.153931
0     -0.064111 -0.167870       2        1   7.334697
11     0.194285  0.106127       3        1   7.203140
3      0.152570 -0.224064       4        1   7.151350
9      0.024396 -0.098464       5        1   7.144855
14     0.138558  0.066654       6        1   6.698597
7      0.057754  0.030049       7        1   6.584839
10     0.097420  0.055676       8        1   6.217615
13     0.125518  0.074674       9        1   6.096012
4     -0.056289 -0.006425      10        1   6.005016
6     -0.009217  0.138438      11        1   5.525305
1     -0.195270  0.090030      12        1   5.372341
12    -0.190382 -0.039503      13        1   5.314516
5     -0.039681 -0.170701      14        1   4.631561
8     -0.125975  0.026294      15        1   4.566225, topic_info=             Term          Freq         Total Category  logprob  loglift
3946    marijuana  21666.000000  21666.000000  Default  30.0000  30.0000
645   cannabinoid   4533.000000   4533.000000  Default  29.0000  29.0000
3945      cannabi  24559.000000  24559.000000  Default  28.0000  28.0000
1582         cure   5677.000000   5677.000000  Default  27.0000  27.0000
912         medic   8214.000000   8214.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1506         rush    278.361297   1603.163082  Topic15  -4.4427   1.3357
470       patient    368.210034   4969.209879  Topic15  -4.1630   0.4841
52         health    289.703117   4205.524889  Topic15  -4.4028   0.4112
3978  cannabisoil    241.302982   1595.491113  Topic15  -4.5856   1.1976
295         brain    216.568578   1464.037576  Topic15  -4.6938   1.1754

[929 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
4132     10  0.962405  abg_marketplac
1094     10  0.998305          accord
5080      9  0.980349      acknowledg
5995      6  0.970282    acquacannabi
4581      7  0.994400             act
...     ...       ...             ...
20        8  0.336757            year
20       12  0.014374            year
20       15  0.028748            year
4269     12  0.990577           yeson
4118     14  0.997455         zionist

[1417 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 12, 4, 10, 15, 8, 11, 14, 5, 7, 2, 13, 6, 9])

In [16]:
# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Obtenez la distribution des sujets pour chaque tweet
tweet_topic_distributions = [lda_model.get_document_topics(dictionary.doc2bow(doc)) for doc in processed_docs]


In [17]:
# Choisissez le sujet dominant pour chaque tweet
dominant_topics = [max(topic_dist, key=lambda x: x[1])[0] for topic_dist in tweet_topic_distributions]

# Ajoutez la colonne 'Dominant Topic' au DataFrame
tweets_df['Dominant Topic'] = dominant_topics


In [18]:
# Créer une Mapping pour les Topics
topic_mapping = {}
for i in range(lda_model.num_topics):
    topic_mapping[i] = 'Topic {}'.format(i+1)


In [19]:
tweets_df['Topic Label'] = [topic_mapping[dominant_topic] for dominant_topic in dominant_topics]


In [20]:
# Affichez les premières lignes du DataFrame
tweets_df.head()


,Fichier,ID,Nb retweet,Nb like,Nb réponses,Nb citations,Hashtags,Texte,Mots,Phrases,Tags POS,Sentiment,Polarité,Subjectivité,Phrases nominales,Texte corrigé,Texte tokénizé,Dominant Topic,Topic Label
0,721916579533950976.json,721916579533950976,0,0,0,0.0,"['#CANCER', '#SPORT', '#PARENTING', '#SCHOOL',...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet...","[Sentence(""Is artificial turf giving athletes ...","[('Is', 'VBZ'), ('artificial', 'JJ'), ('turf',...","Sentiment(polarity=-0.6, subjectivity=1.0)",-0.6,1.0000,"['artificial turf', 'athletes # cancer', 'spor...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet...",4,Topic 5
1,238304774825603072.json,238304774825603072,0,1,0,0.0,"['#CANCER', '#SPORT']",Frankel's victory at York made me feel '20 yea...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma...","[Sentence(""Frankel's victory at York made me f...","[('Frankel', 'NNP'), (""'s"", 'POS'), ('victory'...","Sentiment(polarity=0.5, subjectivity=0.5)",0.5,0.5000,"['frankel', ""'s victory"", 'york', ""'20 years"",...",Fraenkel's victory at Work made me feel '20 ye...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma...",4,Topic 5
2,1007390700726575104.json,1007390700726575104,0,4,1,0.0,"['#SPORT', '#CANCER']",READ THIS! What an absolutely brilliant piece ...,"['READ', 'THIS', 'What', 'an', 'absolutely', '...","[Sentence(""READ THIS!""), Sentence(""What an abs...","[('READ', 'VB'), ('THIS', 'NN'), ('What', 'WP'...","Sentiment(polarity=0.5, subjectivity=0.5375)",0.5,0.5375,"['read this', 'brilliant piece', '# sport trib...",READ THIS! That an absolutely brilliant piece ...,"['READ', 'THIS', '!', 'What', 'an', 'absolutel...",5,Topic 6
3,668965073889697795.json,668965073889697795,0,0,0,0.0,"['#CANCER', '#STROKE', '#SPORT']","MJA InSight News: #Cancer survivors, exercise ...","['MJA', 'InSight', 'News', 'Cancer', 'survivor...","[Sentence(""MJA InSight News: #Cancer survivors...","[('MJA', 'NNP'), ('InSight', 'NNP'), ('News', ...","Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0000,"['mja insight', '# cancer survivors', 'atopic ...","MJA night News: #Cancer survivor, exercise bar...","['MJA', 'InSight', 'News', ':', '#', 'Cancer',...",1,Topic 2
4,1098650711049555968.json,1098650711049555968,2,1,0,0.0,"['#HEALTH', '#CANCER', '#SPORT']",When #health #cancer &amp; #sport are inextric...,"['When', 'health', 'cancer', 'amp', 'sport', '...","[Sentence(""When #health #cancer &amp; #sport a...","[('When', 'WRB'), ('health', 'NN'), ('cancer',...","Sentiment(polarity=0.6000000000000001, subject...",0.6,0.9000,"['# health # cancer', '# sport', 'amazing', 'd...",When #health #cancer &amp; #sport are inextric...,"['When', '#', 'health', '#', 'cancer', '&', 'a...",1,Topic 2
